# Examples of using fuction in "data_description.py"

In [1]:
import pandas as pd
import numpy as np
from data_description import *

# Creation an exmples data frame

In [2]:
frame_size = 100

data = pd.DataFrame({
    'dates_column' : np.datetime64('2017-01-01') + np.random.choice(np.arange(0, 60), frame_size),
    'object_column' : np.random.choice(['lev1', 'lev2', 'lev3'], frame_size),
    'category_column' : pd.Series(np.random.choice(['lev1', 'lev2', 'lev3'], frame_size), dtype = "category"),
    'bool_column' : np.random.choice([True, False], frame_size),
    'float_column' : np.random.normal(5, 1, frame_size),
    'integer_column' : np.random.normal(5, 1, frame_size).astype('int64')
})

# Testing of "get_col_av_values" function

In [4]:
data.apply(get_col_av_values)

dates_column                      [02.01.2017;01.03.2017]
object_column                            lev3, lev2, lev1
category_column                          lev2, lev1, lev3
bool_column                                  [False;True]
float_column       [2.9027224869030603;7.477903928808866]
integer_column                                      [1;7]
dtype: object